## This demo app shows:

[source](https://github.com/facebookresearch/llama-recipes/tree/main)

* How to run Llama2 locally on a Mac using llama-cpp-python and the llama-cpp's quantized Llama2 model
* How to use LangChain to ask Llama general questions
* How to use LangChain to load a recent PDF doc - the Llama2 paper pdf - and ask questions about it. This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama2 be able to answer questions about the data not publicly available when Llama2 was trained, or about your own data. RAG is one way to prevent LLM's hallucination

We start by installing necessary requirements and import packages we will be using in this example.
- [llama-cpp-python](https://github.com/abetlen/llama-cpp-python) a simple Python bindings for [llama.cpp](https://github.com/ggerganov/llama.cpp) library
- pypdf gives us the ability to work with pdfs
- sentence-transformers for text embeddings
- chromadb gives us database capabilities 
- langchain provides necessary RAG tools for this demo

In [1]:
# install all the required packages for the demo
# !CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python
# !pip install pypdf sentence-transformers chromadb langchain

In [1]:
from langchain.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

Next, initialize the langchain `CallBackManager`. This handles callbacks from Langchain and for this example we will use token-wise streaming so the answer gets generated token by token when Llama is answering your question.

In [2]:
# for token-wise streaming so you'll see the answer gets generated token by token when Llama is answering your question
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])


Set up the Llama 2 model. 

Replace `<path-to-llama-gguf-file>` with the path either to your downloaded quantized model file [here](https://drive.google.com/file/d/1afPv3HOy73BE2MoYCgYJvBDeQNa9rZbj/view?usp=sharing), 

or to the `ggml-model-q4_0.gguf` file built with the following commands:

```bash
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
python3 -m pip install -r requirements.txt
python convert.py <path_to_your_downloaded_llama-2-13b_model>
./quantize <path_to_your_downloaded_llama-2-13b_model>/ggml-model-f16.gguf <path_to_your_downloaded_llama-2-13b_model>/ggml-model-q4_0.gguf q4_0

```
For more info see https://python.langchain.com/docs/integrations/llms/llamacpp

In [3]:

llm = LlamaCpp(
    model_path="./models/llama-2-7b-chat.Q4_K_M.gguf",
    temperature=0.0,
    top_p=1,
    n_ctx=6000,
    callback_manager=callback_manager, 
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32       

With the model set up, you are now ready to ask some questions. 

Here is an example of the simplest way to ask the model some general questions.

In [4]:
question = "Who wrote the book Innovator's dilemma?Give me just the name"
answer = llm.invoke(question)
answer

 of the author.
Answer: The book "Innovator's Dilemma" was written by Clayton Christensen.


llama_print_timings:        load time =    6645.95 ms
llama_print_timings:      sample time =       3.12 ms /    29 runs   (    0.11 ms per token,  9288.92 tokens per second)
llama_print_timings: prompt eval time =    7842.82 ms /    21 tokens (  373.47 ms per token,     2.68 tokens per second)
llama_print_timings:        eval time =    3464.82 ms /    28 runs   (  123.74 ms per token,     8.08 tokens per second)
llama_print_timings:       total time =   11386.48 ms /    49 tokens


' of the author.\nAnswer: The book "Innovator\'s Dilemma" was written by Clayton Christensen.'

Alternatively, you can use LangChain's `PromptTemplate` for some flexibility in your prompts and questions.

For more information on LangChain's prompt template visit this [link](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/)

In [5]:
# a more flexible way to ask Llama general questions using LangChain's PromptTemplate and LLMChain
prompt = PromptTemplate.from_template(
    "who wrote {book}?"
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.invoke("innovator's dilemma")

Llama.generate: prefix-match hit




The Innovator's Dilemma is a book written by Clayton Christensen, a Harvard Business School professor and leading authority on innovation and business strategy. The book was first published in 1997 and has since become a classic in the field of entrepreneurship and business management.


llama_print_timings:        load time =    6645.95 ms
llama_print_timings:      sample time =       6.98 ms /    67 runs   (    0.10 ms per token,  9596.10 tokens per second)
llama_print_timings: prompt eval time =    1991.45 ms /    11 tokens (  181.04 ms per token,     5.52 tokens per second)
llama_print_timings:        eval time =    5241.84 ms /    66 runs   (   79.42 ms per token,    12.59 tokens per second)
llama_print_timings:       total time =    7374.04 ms /    77 tokens


Now, let's see how Llama2 hallucinates, because it did not have knowledge about Llama2 at the time it was trained. 
By default it behaves like a know-it-all expert who will not say "I don't know".

In [7]:
prompt = PromptTemplate.from_template(
    "Describe {item} in bullet points."
)
chain = LLMChain(llm=llm, prompt=prompt)
answer = chain.invoke("Large Language Model")

Llama.generate: prefix-match hit




•	Large language models are artificial intelligence (AI) systems that are trained on vast amounts of text data to generate language outputs that are coherent and natural-sounding.
•	These models have become increasingly popular in recent years due to their ability to generate text that is often indistinguishable from human-written content.
•	Large language models can be used for a variety of applications, including language translation, text summarization, and chatbots.
•	The most popular large language models include transformer-based models such as BERT, RoBERTa, and XLNet.
•	These models are trained using deep learning techniques and can learn to recognize patterns in language at a scale that was previously not possible.
•	Large language models have the potential to revolutionize many industries, including customer service, marketing, and entertainment.
•	However, they also raise important ethical and social questions about the impact of AI on society.
•	For example, large languag


llama_print_timings:        load time =    6645.95 ms
llama_print_timings:      sample time =      26.85 ms /   256 runs   (    0.10 ms per token,  9533.03 tokens per second)
llama_print_timings: prompt eval time =   36881.16 ms /    11 tokens ( 3352.83 ms per token,     0.30 tokens per second)
llama_print_timings:        eval time =   35891.98 ms /   255 runs   (  140.75 ms per token,     7.10 tokens per second)
llama_print_timings:       total time =   38167.88 ms /   266 tokens
